In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import keras
import tensorflow as tf
import numpy as np
from numpy  import expand_dims
import cv2
from matplotlib import pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
np.random.seed(15)

Using TensorFlow backend.


In [16]:
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# 검증 및 테스트 이미지는 augmentation을 적용하지 않음(이미지 원본을 사용)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# 이미지를 배치 단위로 불러와 줄 generator입니다.
train_generator = train_datagen.flow_from_directory(
        'clean-dataset/train', 
        target_size=(150, 150), 
        batch_size=batch_size,
        class_mode='binary') 

validation_generator = validation_datagen.flow_from_directory(
        'clean-dataset/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        'clean-dataset/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 600 images belonging to 3 classes.
Found 150 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


In [17]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [18]:
model.fit_generator(
        train_generator,
        steps_per_epoch=600 // batch_size,
        epochs=50,  #50 정도는 해야 accuracy가 0.8까지는 나옴
        validation_data=validation_generator,
        validation_steps=150 // 16)     

model.save("pumpkin_tomato_watermelon.h5")

Epoch 1/50
37/37 [==============================] - 5s 139ms/step - loss: 0.9120 - accuracy: 0.5873 - val_loss: 0.6231 - val_accuracy: 0.5764
Epoch 2/50
37/37 [==============================] - 5s 138ms/step - loss: 0.6469 - accuracy: 0.7483 - val_loss: 0.8288 - val_accuracy: 0.8060
Epoch 3/50
37/37 [==============================] - 5s 140ms/step - loss: 0.5115 - accuracy: 0.8185 - val_loss: 0.5750 - val_accuracy: 0.8731
Epoch 4/50
37/37 [==============================] - 5s 139ms/step - loss: 0.4553 - accuracy: 0.8305 - val_loss: 0.6141 - val_accuracy: 0.7761
Epoch 5/50
37/37 [==============================] - 5s 139ms/step - loss: 0.4117 - accuracy: 0.8459 - val_loss: 0.5746 - val_accuracy: 0.9030
Epoch 6/50
37/37 [==============================] - 5s 138ms/step - loss: 0.3721 - accuracy: 0.8510 - val_loss: 0.2949 - val_accuracy: 0.8806
Epoch 7/50
37/37 [==============================] - 5s 138ms/step - loss: 0.3650 - accuracy: 0.8630 - val_loss: 0.0319 - val_accuracy: 0.8731
Epoch 

In [19]:
# 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator( test_generator,   steps = 150//16)
print(scores[1])

-- Evaluate --
0.9097222089767456
